# Bouncing Pendulum

This example demonstrates a hybrid system combining continuous pendulum dynamics with discrete bounce events. The pendulum swings until it hits the ground (zero angle), at which point it bounces back with reduced energy.

You can also find this example as a single file in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/examples/examples_event/example_bouncing_pendulum.py).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style
plt.style.use('../pathsim_docs.mplstyle')

from pathsim import Simulation, Connection
from pathsim.blocks import Integrator, Amplifier, Function, Adder, Scope
from pathsim.solvers import RKCK54
from pathsim.events import ZeroCrossing
from pathsim.optim import Value

## System Dynamics

The mathematical pendulum is governed by the nonlinear differential equation:

$$\ddot{\phi} = -\frac{g}{l} \sin(\phi)$$

where:
- $\phi$ is the angle from vertical
- $g$ is gravitational acceleration
- $l$ is the pendulum length

The bounce event occurs when $\phi = 0$ (pendulum hits the ground), at which point the angular velocity reverses with a loss factor.

In [ ]:
# Initial angle and angular velocity
phi0, omega0 = 0.99*np.pi, 0.0

# Parameters (gravity, length)
g, l = 9.81, 1

# Bounceback coefficient (for sensitivity analysis)
b = Value(0.9)

## Block Diagram Construction

We construct the system from basic blocks:

In [ ]:
# Blocks that define the system
In1 = Integrator(omega0)  # angular acceleration -> angular velocity
In2 = Integrator(phi0)    # angular velocity -> angle
Amp = Amplifier(-g/l)     # gravity term
Fnc = Function(np.sin)    # nonlinearity
Sco = Scope(labels=[r"$\omega$", r"$\phi$"])

blocks = [In1, In2, Amp, Fnc, Sco]

# Connections between the blocks
connections = [
    Connection(In1, In2, Sco[0]),
    Connection(In2, Fnc, Sco[1]),
    Connection(Fnc, Amp),
    Connection(Amp, In1)
]

In [ ]:
# Event function for zero crossing detection
def func_evt(t):
    *_, ph = In2()
    return ph

# Action function for state transformation
def func_act(t):
    *_, om = In1()
    *_, ph = In2()
    In1.engine.set(-om*b)  # reverse velocity with energy loss
    In2.engine.set(abs(ph))  # ensure angle stays positive

# Events (zero crossing)
E1 = ZeroCrossing(
    func_evt=func_evt,
    func_act=func_act,
    tolerance=1e-6
)

events = [E1]

In [ ]:
# Simulation instance from the blocks and connections
Sim = Simulation(
    blocks,
    connections,
    events,
    dt=0.1,
    log=True,
    Solver=RKCK54,
    tolerance_lte_abs=1e-8,
    tolerance_lte_rel=1e-6
)

Now let's run the simulation:

In [ ]:
Sim.run(duration=15)

## Results

Let's plot the angular velocity and angle over time, marking the bounce events:

In [ ]:
# Plot the results directly from the scope
fig, ax = Sco.plot(lw=2)

plt.show()

The plot shows:
- The angle oscillates between 0 and some maximum value that decreases over time
- The angular velocity reverses sign at each bounce (vertical dashed lines)
- Energy is progressively lost with each bounce, leading to smaller oscillations

## Sensitivity Analysis

Since we wrapped the bounceback coefficient in a `Value` instance, we can extract sensitivities of the system response with respect to this parameter:

In [ ]:
# Read the recordings from the scope
time, [om, ph] = Sco.read()

# Extract sensitivities
dom_db = Value.der(om, b)
dph_db = Value.der(ph, b)

# Plot sensitivities
fig, ax = plt.subplots(figsize=(8, 4), tight_layout=True, dpi=120)

ax.plot(time, dom_db, lw=2, c="tab:red", label=r"$\partial \omega / \partial b$")
ax.plot(time, dph_db, lw=2, c="tab:blue", label=r"$\partial \phi / \partial b$")

ax.set_xlabel("time [s]")
ax.legend()
ax.grid(True)

plt.show()